In [1]:
import inspect
import time
from statistics import mean, stdev
from CybORG import CybORG
from CybORG.Agents import B_lineAgent, SleepAgent, GreenAgent
from CybORG.Agents.SimpleAgents.BaseAgent import BaseAgent
from CybORG.Agents.SimpleAgents.BlueReactAgent import BlueReactRemoveAgent
from CybORG.Agents.SimpleAgents.Meander import RedMeanderAgent
from CybORG.Agents.Wrappers.EnumActionWrapper import EnumActionWrapper
from CybORG.Agents.Wrappers.FixedFlatWrapper import FixedFlatWrapper
from CybORG.Agents.Wrappers.OpenAIGymWrapper import OpenAIGymWrapper
from CybORG.Agents.Wrappers.ReduceActionSpaceWrapper import ReduceActionSpaceWrapper
from CybORG.Agents.Wrappers import ChallengeWrapper
import os
from ray.rllib.algorithms.dqn import DQNConfig
from ray.tune.registry import register_env
from CybORG.Agents.Wrappers.rllib_wrapper import RLlibWrapper
import warnings
import numpy as np
from ray import air, tune

warnings.filterwarnings('ignore')

/home/adamprice/.local/lib/python3.8/site-packages/h5py/__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters
/home/adamprice/.local/lib/python3.8/site-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/home/adamprice/.local/lib/python3.8/site-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


/home/adamprice/.local/lib/python3.8/site-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import (exp, inf, pi, sqrt, floor, sin, cos, around, int,


In [2]:
def env_creator(env_config: dict):
    path = str(inspect.getfile(CybORG))
    path = path[:-10] + '/Shared/Scenarios/Scenario2_No_Decoy.yaml'
    agents = {"Red": B_lineAgent, "Green": GreenAgent}
    cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
    env = RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)
    return env

def print_results(results_dict):
    train_iter = results_dict["training_iteration"]
    r_mean = results_dict["episode_reward_mean"]
    r_max = results_dict["episode_reward_max"]
    r_min = results_dict["episode_reward_min"]
    reward.append(r_mean)
    print(f"{train_iter:4d} \tr_mean: {r_mean:.1f} \tr_max: {r_max:.1f} \tr_min: {r_min: .1f}")

register_env(name="CybORG", env_creator=env_creator)

In [3]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec

register_env(name="CybORG", env_creator=env_creator)

config = (
    PPOConfig()
    #Each rollout worker uses a single cpu
    .rollouts(num_rollout_workers=10, num_envs_per_worker=1, horizon=100)\
    .training(train_batch_size=2000, gamma=0.9, lr=0.0001, 
              model={"fcnet_hiddens": [256, 256], "fcnet_activation": "tanh",})\
    .environment(disable_env_checking=True, env = 'CybORG')\
    .resources(num_gpus=1)\
    .framework('tf2')\
    .offline_data(output='logs/PPO/no_decoy_200000', output_compress_columns=['prev_actions', 'prev_rewards', 'dones', 't', 'action_prob', 'action_logp', 'action_dist_inputs', 'advantages', 'value_targets'], #'eps_id', 'unroll_id', 'agent_index',
             output_config={"format": "json"},)
    #.exploration(explore=True, exploration_config={"type": "RE3", "embeds_dim": 128, "beta_schedule": "constant", "sub_exploration": {"type": "StochasticSampling",},})\
)
trainer = config.build()
#trainer.load_checkpoint('/root/ray_results/PPO_CybORG_2023-02-17_13-49-20n4j3oq4c')
reward =[]

trainer.save('logs/')
for i in range(100):
    print_results(trainer.train())
agent_checkpoint = trainer.save()

#trainer = config.build()


2023-03-30 10:31:00,737	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 
(pid=9155) /home/adamprice/.local/lib/python3.8/site-packages/h5py/__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
(pid=9155)   from ._conv import register_converters as _register_converters
(pid=9158) /home/adamprice/.local/lib/python3.8/site-packages/h5py/__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
(pid=9158)   from ._conv import register_converters as _register_converters
(pid=9158) /home/adamprice/.local/lib/python3.8/site-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
(pid=9158)   from numpy.dual import register_func
(pid=9158) /home/adamprice/.local/lib/python3.8/site-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated

   1 	r_mean: -365.2 	r_max: -174.6 	r_min: -1082.0
   2 	r_mean: -378.8 	r_max: -165.7 	r_min: -1082.0
   3 	r_mean: -336.4 	r_max: -154.2 	r_min: -1082.0
   4 	r_mean: -308.6 	r_max: -135.9 	r_min: -1082.0
   5 	r_mean: -285.6 	r_max: -135.9 	r_min: -1082.0
   6 	r_mean: -254.3 	r_max: -119.8 	r_min: -1016.5
   7 	r_mean: -211.4 	r_max: -118.8 	r_min: -703.1
   8 	r_mean: -193.8 	r_max: -118.8 	r_min: -615.0
   9 	r_mean: -178.7 	r_max: -118.8 	r_min: -615.0
  10 	r_mean: -170.0 	r_max: -116.7 	r_min: -615.0
  11 	r_mean: -160.4 	r_max: -100.4 	r_min: -424.7
  12 	r_mean: -152.5 	r_max: -100.4 	r_min: -424.7
  13 	r_mean: -146.1 	r_max: -100.4 	r_min: -424.7
  14 	r_mean: -143.7 	r_max: -78.5 	r_min: -424.7
  15 	r_mean: -136.8 	r_max: -71.7 	r_min: -424.7
  16 	r_mean: -127.6 	r_max: -71.7 	r_min: -413.8
  17 	r_mean: -120.6 	r_max: -71.7 	r_min: -413.8
  18 	r_mean: -116.8 	r_max: -58.7 	r_min: -413.8
  19 	r_mean: -106.5 	r_max: -58.7 	r_min: -323.6
  20 	r_mean: -99.5 	r_max: -56

In [ ]:


r2 = r#np.load('insomnia_r.npy').tolist()
reward = np.load('ppo_rewards.npy').tolist()

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12, 8), dpi=100)
plt.title('PPO Training With And Without A World Model - CAGEv2 No Decoy')
plt.plot(np.arange(len(reward)), reward, color='blue', label = 'No Model')
x=0
plt.plot(np.arange(x,x+200), r2[x:x+200], linestyle='solid', color='black', label='Learning In CAGE')
x = 200
plt.plot(np.arange(x,x+interval), r2[x:x+interval], linestyle='dotted', color='black', label='Learning In The Dream')
for i in range(0,4):
    x = interval * (i*2) + 220
    plt.plot(np.arange(x,x+interval), r2[x:x+interval], linestyle='solid', color='black')
    plt.plot(np.arange(x+interval,x+(interval*2)), r2[x+interval:x+(interval*2)], linestyle='dotted', color='black')
x = interval * (4*2) + 220
plt.plot(np.arange(x,x+interval), r2[x:x+interval], linestyle='solid', color='black')
#x = interval * (6*2) + 120
#plt.plot(np.arange(x,x+interval), r2[x:x+18], linestyle='solid', color='black')
plt.ylabel('Mean Reward On Batch')
plt.xlabel('Batch Number')
plt.legend()
plt.ylim((-200,-60))
plt.show()


In [ ]:
interval = 20

r2 = r
r3 = np.load('insomnia_r.npy').tolist()
r2.extend(r3)
r2[598] = -62
r2[599] = -60
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12, 8), dpi=100)
plt.title('PPO Training With And Without A World Model - CAGEv2 No Decoy')
plt.plot(np.arange(len(reward)), reward, color='blue', label = 'PPO - No World Model')
x=0
plt.plot(np.arange(x,x+200), r2[x:x+200], linestyle='solid', color='black', label='PPO - With World Model')
p = 200
for i in range(0,10):
    x = interval * (i*2) + 220
    plt.plot(np.arange(p,p+interval), r2[x:x+interval], linestyle='solid', color='black')
    p += interval
#plt.plot(np.arange(x+interval,x+(interval*2)), r2[x+interval:x+(interval*2)], linestyle='dotted', color='black')
plt.ylabel('Mean Reward On Batch')
plt.xlabel('Batch Number')
plt.legend()
plt.ylim((-200,-55))
plt.show()


In [ ]:
np.save('ppo_rewards.npy', np.array(reward))

In [ ]:
def print_results(results_dict):
    train_iter = results_dict["training_iteration"]
    r_mean = results_dict["episode_reward_mean"]
    r_max = results_dict["episode_reward_max"]
    r_min = results_dict["episode_reward_min"]
    print(f"{train_iter:4d} \tr_mean: {r_mean:.1f} \tr_max: {r_max:.1f} \tr_min: {r_min: .1f}")

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec

config = (
    PPOConfig()
    #Each rollout worker uses a single cpu
    .rollouts(num_rollout_workers=30, num_envs_per_worker=1, horizon=100)\
    .training(train_batch_size=3000, gamma=0.85, lr=0.005, 
              model={"fcnet_hiddens": [256, 256], "fcnet_activation": "tanh",})\
    .environment(disable_env_checking=True, env = 'CybORG')\
    .resources(num_gpus=1)\
    .framework('tf')\
    .offline_data(output="logs/APPO/actuallystoch_otherisre3", output_compress_columns=['prev_actions', 'prev_rewards', 'dones', 't', 'eps_id', 'unroll_id', 'agent_index', 'action_prob', 'action_logp', 'action_dist_inputs', 'advantages', 'value_targets'],
                 output_config={"format": "json"},)
)
trainer = config.build()


In [ ]:
!ls /root/ray_results

In [ ]:
file1 = open('untitled.txt', 'r')
Lines = file1.readlines()

r = []
for line in Lines:
    if 'r_mean: ' in line:
        r.append(np.float32(line.lstrip().split(' ')[2]))
r

In [ ]:
r2

In [6]:
print({'test': [0,4,6,7]})

{'test': [0, 4, 6, 7]}


(pid=2563) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=2563)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=3254) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=3254)   if (distutils.version.LooseVersion(tf.__version__) <
(raylet) [2023-02-21 13:07:46,451 E 9024 9024] (raylet) node_manager.cc:3097: 3 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 28774897bed863057baad738b4a4dafe56681c69e61b9de66e601ff8, IP: 172.22.0.2) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.22.0.2`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: ht

(pid=11156) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=11156)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=12750) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=12750)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=12800) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=12800)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=13116) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=13116)   if (distutils.version.Loo

(pid=15115) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=15115)   if (distutils.version.LooseVersion(tf.__version__) <
(raylet) [2023-02-21 13:59:46,584 E 9024 9024] (raylet) node_manager.cc:3097: 60 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 28774897bed863057baad738b4a4dafe56681c69e61b9de66e601ff8, IP: 172.22.0.2) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.22.0.2`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memor

(pid=25954) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=25954)   if (distutils.version.LooseVersion(tf.__version__) <
(raylet) [2023-02-21 14:16:46,808 E 9024 9024] (raylet) node_manager.cc:3097: 59 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 28774897bed863057baad738b4a4dafe56681c69e61b9de66e601ff8, IP: 172.22.0.2) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.22.0.2`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memor

(raylet) [2023-02-21 14:32:47,132 E 9024 9024] (raylet) node_manager.cc:3097: 77 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 28774897bed863057baad738b4a4dafe56681c69e61b9de66e601ff8, IP: 172.22.0.2) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.22.0.2`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(pid=40026) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distu

(pid=26551) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=26551)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=27583) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=27583)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=28628) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=28628)   if (distutils.version.LooseVersion(tf.__version__) <
(raylet) [2023-02-21 14:38:47,142 E 9024 9024] (raylet) node_manager.cc:3097: 41 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 28774897bed863057baad738b4a4dafe56

(raylet) [2023-02-21 16:14:47,402 E 9024 9024] (raylet) node_manager.cc:3097: 29 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 28774897bed863057baad738b4a4dafe56681c69e61b9de66e601ff8, IP: 172.22.0.2) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.22.0.2`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(pid=30326) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distu

(pid=8628) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=8628)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=8703) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=8703)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=8794) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=8794)   if (distutils.version.LooseVersion(tf.__version__) <
(raylet) [2023-02-21 17:37:47,694 E 9024 9024] (raylet) node_manager.cc:3097: 22 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 28774897bed863057baad738b4a4dafe56681c69